In [1]:
import socket
import threading

In [2]:
dest_MTU = {
    'Z' : 13,
    'T' : 15
}

In [3]:
d1 = {}

In [ ]:
HOST = '192.168.1.9'
PORT = 65432

ADDR = (HOST,PORT)

server_socket = socket.socket(family = socket.AF_INET, type = socket.SOCK_STREAM)
server_socket.bind(ADDR)
print("Server Started listening")

def handle_client(conn, addr):
    cnt = 0
    print(f"[NEW CONNECTION] {addr} connected.")
    data = conn.recv(5000).decode("utf-8")
    output = []
    print("Message from Client: ", data)
    x = data.split(' ')
    if x[0] == "MTUCONFIG:":
        dest_MTU[x[1]] = x[2]
        s1 = "MTU Changed Successfully"
        print("Data sent to client:",s1)
        s1 = s1.encode("utf-8")
        conn.send(s1)
    elif data.isalpha() and len(data) == 1:
        send_d = d1[data]
        conn.send(send_d)
    else:
        IP_list = data.split(":")
        length_of_data = int(IP_list[0])
        dest = str(IP_list[1])
        more_frag = int(IP_list[3])
        frag_offset = int(IP_list[4])
        print("FRAGMENT OFFSET :",)
        msg_to_client = str(IP_list[5])
        length_of_message = len(msg_to_client)
        start = 0
        index = int(dest_MTU[dest]) - 5
        mf = 0
        fo = 0
        remain = length_of_message
        #while remain >= (13 - 5):
        while index <= length_of_message:
            list1 = []
            str1 = ""
            str1 += msg_to_client[start: index]
            list1.append(str(dest_MTU[dest]))
            list1.append(str(IP_list[1]))
            list1.append(str(IP_list[2]))                   
            list1.append(str(IP_list[3]))      
            list1.append(str(IP_list[4]))      
            list1.append(str(str1))
            output.append(list1)
            start = index
            index = index + (int(dest_MTU[dest]) - 5)
            #remain = length_of_message - (13 - 5)
        list1 = []
        str1 = ""
        str1 += msg_to_client[start: length_of_message]
        list1.append(str(length_of_message - start + 5))
        list1.append(str(IP_list[1]))
        list1.append(str(IP_list[2]))                    
        list1.append(str(IP_list[3]))       
        list1.append(str(IP_list[4]))      
        list1.append(str(str1))
        output.append(list1)
        #print(output)
        string_l = [','.join(i) for i in output]
        output_s = '|'.join(string_l)
        output_s = output_s.encode("utf-8")
        conn.sendall(output_s)
        print("Data sent to client:",output_s)
        data_z = conn.recv(5000)
        conn.send(data_z)
        d1[dest] = output_s
        print(d1)
        conn.send("Server Msg received".encode("utf-8"))
    

    conn.close()
        

def start():
    server_socket.listen(6)
    print("[LISTENING] Server is listening")
    while True:
        conn, addr = server_socket.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()

print("[STARTING] server is starting...")
start()

Server Started listening
[STARTING] server is starting...
[LISTENING] Server is listening
[NEW CONNECTION] ('192.168.1.9', 62014) connected.
Message from Client:  23:Z:0:0:0:123456789012345678
Data sent to client: b'13,Z,0,0,0,12345678|13,Z,0,0,0,90123456|7,Z,0,0,0,78'
{'Z': b'13,Z,0,0,0,12345678|13,Z,0,0,0,90123456|7,Z,0,0,0,78'}
[NEW CONNECTION] ('192.168.1.9', 62513) connected.
Message from Client:  Z
[NEW CONNECTION] ('192.168.1.9', 62514) connected.
Message from Client:  23:T:0:0:0:123456789012345678
Data sent to client: b'15,T,0,0,0,1234567890|13,T,0,0,0,12345678'
{'Z': b'13,Z,0,0,0,12345678|13,Z,0,0,0,90123456|7,Z,0,0,0,78', 'T': b'15,T,0,0,0,1234567890|13,T,0,0,0,12345678'}
[NEW CONNECTION] ('192.168.1.9', 57617) connected.
Message from Client:  T
[NEW CONNECTION] ('192.168.1.9', 57618) connected.
Message from Client:  MTUCONFIG: X 18
Data sent to client: MTU Changed Successfully
[NEW CONNECTION] ('192.168.1.9', 51599) connected.
Message from Client:  23:X:0:0:0:123456789012345